# 02 — Spatial Weights & Moran's I
This notebook demonstrates KNN weights and global spatial autocorrelation.

In [ ]:
# --- Course bootstrap: installs + imports + settings ---
# System deps for rtree
!apt-get -qq install -y libspatialindex-dev

# Python deps (pin versions for stability)
%pip -q install geopandas==0.14.4 shapely==2.0.4 pyproj==3.6.1 rtree==1.2.0 \  libpysal==4.12.0 esda==2.6.0 splot==1.1.7 mapclassify==2.6.1 \  pyogrio==0.9.0 contextily==1.6.0 folium==0.17.0 ipywidgets==8.1.3

# Imports
import warnings; warnings.filterwarnings("ignore")
import pandas as pd, numpy as np, geopandas as gpd
from libpysal import weights
from esda import Moran
import contextily as cx
from pyogrio import read_dataframe

# Display settings
pd.set_option("display.max_rows", 8)
print("Environment ready.")

In [ ]:
# Load data and compute a statistic for Moran's I
import geopandas as gpd, numpy as np
gdf = gpd.read_file('data/neighborhoods.geojson').to_crs(3857)

# Simple numeric variable
y = gdf['median_income'].astype(float).values

# 8-nearest neighbors (row-standardized)
from libpysal import weights
w = weights.KNN.from_dataframe(gdf, k=8)
w.transform = "R"

from esda import Moran
mi = Moran(y, w)
print("Moran's I:", round(mi.I, 4), "| p_sim:", mi.p_sim)

In [ ]:
# Moran scatterplot (quick and simple)
import pandas as pd
import matplotlib.pyplot as plt
from libpysal.weights import spatial_lag

gdf['y_std'] = (gdf['median_income'] - gdf['median_income'].mean())
gdf['lag_y_std'] = spatial_lag.lag_spatial(w, gdf['y_std'])

plt.figure(figsize=(5,5))
plt.scatter(gdf['y_std'], gdf['lag_y_std'])
# fit line
m, b = np.polyfit(gdf['y_std'], gdf['lag_y_std'], 1)
xs = np.linspace(gdf['y_std'].min(), gdf['y_std'].max(), 100)
plt.plot(xs, m*xs+b)
plt.axhline(0, color='k', alpha=0.3)
plt.axvline(0, color='k', alpha=0.3)
plt.title("Moran Scatterplot")
plt.xlabel("Attribute (standardized)")
plt.ylabel("Spatial Lag")
plt.show()